## Algoritmo de automatización 



In [1]:
import json
import numpy as np
import logging
import os
import math
import copy
import random
import string
from datetime import datetime, timezone
from typing import Dict, List, Any, Optional, Tuple, Set

# --- Configuration ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# --- File Paths ---
RELAY_PAIRS_PATH = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs.json"
OPTIMIZED_SETTINGS_OUTPUT_PATH = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/raw/optimized_relay_values_v8.json"

# --- Constants ---
K = 0.14
N = 0.02
CTI = 0.2
# Optimization Bounds and Parameters
MIN_TDS = 0.05
MAX_TDS = 1
MIN_PICKUP = 0.05
MAX_PICKUP_FACTOR = 0.7
MAX_TIME = 20.0
MAX_ITERATIONS = 250
# Convergence Targets (Normal Mode)
TARGET_TMT = -0.005
MIN_ALLOWED_INDIVIDUAL_MT = -0.009
# Convergence Targets (Island Mode)
TARGET_TMT_ISLAND = -0.05  # More relaxed for island mode
MIN_ALLOWED_INDIVIDUAL_MT_ISLAND = -0.02  # More relaxed for island mode
CONVERGENCE_THRESHOLD_TMT = 0.005
# Objective Function Weights (Normal Mode)
W_TIME = 0.1
W_MT = 15.0
W_PICKUP_DIFF = 0.0
# Objective Function Weights (Island Mode)
W_TIME_ISLAND = 0.5  # Higher to focus on minimizing operation time
W_MT_ISLAND = 5.0  # Lower to reduce penalty on miscoordination
# Adjustment Step Sizes & Factors (Normal Mode)
AGGRESSIVE_MT_THRESHOLD = -CTI * 0.75
AGGRESSIVE_TDS_BACKUP_FACTOR = 1.15
AGGRESSIVE_TDS_MAIN_FACTOR = 0.90
AGGRESSIVE_PICKUP_BACKUP_FACTOR = 1.05
NORMAL_TDS_BACKUP_ADD = 0.02
NORMAL_TDS_MAIN_SUB = 0.01
NORMAL_PICKUP_BACKUP_FACTOR = 1.01
NORMAL_PICKUP_MAIN_FACTOR = 0.99
# Adjustment Step Sizes & Factors (Island Mode - More Conservative)
AGGRESSIVE_TDS_BACKUP_FACTOR_ISLAND = 1.05  # Smaller adjustment
AGGRESSIVE_TDS_MAIN_FACTOR_ISLAND = 0.95  # Smaller adjustment
AGGRESSIVE_PICKUP_BACKUP_FACTOR_ISLAND = 1.02  # Smaller adjustment
NORMAL_TDS_BACKUP_ADD_ISLAND = 0.01  # Smaller adjustment
NORMAL_TDS_MAIN_SUB_ISLAND = 0.005  # Smaller adjustment
NORMAL_PICKUP_BACKUP_FACTOR_ISLAND = 1.005  # Smaller adjustment
NORMAL_PICKUP_MAIN_FACTOR_ISLAND = 0.995  # Smaller adjustment
# Island Mode Detection
ISLAND_MODE_TMT_THRESHOLD = -10.0  # Threshold to identify island mode scenarios
EXCLUDED_RELAYS = {f"R{i}" for i in [2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 
                                     37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52]}

# --- Helper Functions ---
def load_json_file(file_path: str) -> Optional[Any]:
    try:
        with open(file_path, 'r') as file: data = json.load(file)
        logger.info(f"Archivo cargado: {file_path}"); return data
    except FileNotFoundError: logger.error(f"No encontrado: {file_path}"); return None
    except json.JSONDecodeError as e: logger.error(f"JSON inválido: {file_path}: {e}"); return None
    except Exception as e: logger.error(f"Error carga {file_path}: {e}"); return None

def calculate_operation_time(I_shc: float, I_pi: float, TDS: float) -> Optional[float]:
    if I_pi <= 0 or I_shc <= 0 or TDS < MIN_TDS or TDS > MAX_TDS:
        return None
    if I_pi > I_shc * MAX_PICKUP_FACTOR:
        return MAX_TIME
    M = I_shc / I_pi
    if M <= 1.0:
        return MAX_TIME
    try:
        denominator = M**N - 1
        if abs(denominator) < 1e-9:
            return None
        time = TDS * (K / denominator)
        if not np.isfinite(time) or time <= 0:
            return None
        return min(time, MAX_TIME)
    except (OverflowError, ValueError):
        return None
    except Exception as e:
        logger.error(f"Excepción inesperada en calc_op_time (I_shc={I_shc}, I_pi={I_pi}, TDS={TDS}): {e}")
        return None

def group_data_by_scenario(relay_pairs_data: List[Dict]) -> Dict[str, Dict[str, Any]]:
    scenario_map: Dict[str, Dict[str, Any]] = {}
    processed_pairs_count = 0
    skipped_pairs_count = 0
    island_scenarios = set()
    
    # First pass: Identify island mode scenarios based on TMT
    for pair_entry in relay_pairs_data:
        scenario_id = pair_entry.get("scenario_id")
        tmt = pair_entry.get("TMT")
        if scenario_id and isinstance(tmt, (int, float)) and tmt < ISLAND_MODE_TMT_THRESHOLD:
            island_scenarios.add(scenario_id)
            logger.info(f"Escenario {scenario_id} identificado como modo isla (TMT={tmt:.4f} < {ISLAND_MODE_TMT_THRESHOLD})")

    # Second pass: Process pairs, excluding pairs involving EXCLUDED_RELAYS in island mode scenarios
    for i, pair_entry in enumerate(relay_pairs_data):
        scenario_id = pair_entry.get("scenario_id")
        main_relay_info = pair_entry.get('main_relay')
        backup_relay_info = pair_entry.get('backup_relay')
        if not scenario_id or not isinstance(main_relay_info, dict) or not isinstance(backup_relay_info, dict):
            skipped_pairs_count += 1
            continue
        if scenario_id not in scenario_map:
            scenario_map[scenario_id] = {"pairs_info": [], "initial_settings": {}, "relays": set(), "is_island_mode": scenario_id in island_scenarios}
        main_relay = main_relay_info.get('relay')
        backup_relay = backup_relay_info.get('relay')
        I_shc_main = main_relay_info.get('Ishc')
        I_shc_backup = backup_relay_info.get('Ishc')

        # Skip pairs involving EXCLUDED_RELAYS in island mode scenarios
        if scenario_id in island_scenarios and (main_relay in EXCLUDED_RELAYS or backup_relay in EXCLUDED_RELAYS):
            logger.debug(f"Omitiendo par {main_relay}/{backup_relay} en {scenario_id} (modo isla, relé excluido)")
            skipped_pairs_count += 1
            continue

        if not (main_relay and backup_relay and
                isinstance(I_shc_main, (int, float)) and I_shc_main > 0 and
                isinstance(I_shc_backup, (int, float)) and I_shc_backup > 0):
            skipped_pairs_count += 1
            continue

        processed_pairs_count += 1
        scenario_map[scenario_id]['relays'].add(main_relay)
        scenario_map[scenario_id]['relays'].add(backup_relay)
        initial_settings_scenario = scenario_map[scenario_id]['initial_settings']
        for r_name, r_info in [(main_relay, main_relay_info), (backup_relay, backup_relay_info)]:
            if r_name not in initial_settings_scenario:
                tds = r_info.get('TDS')
                pickup = r_info.get('pick_up')
                if isinstance(tds, (int, float)) and isinstance(pickup, (int, float)):
                    initial_settings_scenario[r_name] = {
                        'TDS_initial': float(tds),
                        'pickup_initial': float(pickup)
                    }
        scenario_map[scenario_id]['pairs_info'].append({
            "main_relay": main_relay,
            "backup_relay": backup_relay,
            "I_shc_main": float(I_shc_main),
            "I_shc_backup": float(I_shc_backup)
        })

    logger.info(f"Datos agrupados por escenario. Pares procesados: {processed_pairs_count}, Pares omitidos: {skipped_pairs_count}")
    return scenario_map

def run_scenario_optimization(
    scenario_id: str,
    pairs_info: List[Dict],
    initial_settings: Dict[str, Dict[str, float]],
    relays_in_scenario: Set[str],
    is_island_mode: bool
) -> Dict[str, Dict[str, float]]:
    logger.info(f"--- Iniciando optimización para {scenario_id} ---")
    if not pairs_info:
        logger.warning(f"({scenario_id}) No hay pares válidos.")
        return {}

    relay_settings = {}
    default_pickup = MIN_PICKUP * 1.5
    for relay in relays_in_scenario:
        if relay in initial_settings:
            relay_settings[relay] = {
                "TDS": MIN_TDS,
                "pickup": max(MIN_PICKUP, initial_settings[relay]['pickup_initial'])
            }
        else:
            logger.warning(f"({scenario_id}) Config inicial (pickup) no encontrada para '{relay}'. Usando defecto.")
            relay_settings[relay] = {"TDS": MIN_TDS, "pickup": default_pickup}

    last_tmt = float('inf')
    no_improvement_streak = 0

    # Select optimization parameters based on mode
    w_time = W_TIME_ISLAND if is_island_mode else W_TIME
    w_mt = W_MT_ISLAND if is_island_mode else W_MT
    target_tmt = TARGET_TMT_ISLAND if is_island_mode else TARGET_TMT
    min_allowed_individual_mt = MIN_ALLOWED_INDIVIDUAL_MT_ISLAND if is_island_mode else MIN_ALLOWED_INDIVIDUAL_MT
    aggressive_tds_backup_factor = AGGRESSIVE_TDS_BACKUP_FACTOR_ISLAND if is_island_mode else AGGRESSIVE_TDS_BACKUP_FACTOR
    aggressive_tds_main_factor = AGGRESSIVE_TDS_MAIN_FACTOR_ISLAND if is_island_mode else AGGRESSIVE_TDS_MAIN_FACTOR
    aggressive_pickup_backup_factor = AGGRESSIVE_PICKUP_BACKUP_FACTOR_ISLAND if is_island_mode else AGGRESSIVE_PICKUP_BACKUP_FACTOR
    normal_tds_backup_add = NORMAL_TDS_BACKUP_ADD_ISLAND if is_island_mode else NORMAL_TDS_BACKUP_ADD
    normal_tds_main_sub = NORMAL_TDS_MAIN_SUB_ISLAND if is_island_mode else NORMAL_TDS_MAIN_SUB
    normal_pickup_backup_factor = NORMAL_PICKUP_BACKUP_FACTOR_ISLAND if is_island_mode else NORMAL_PICKUP_BACKUP_FACTOR
    normal_pickup_main_factor = NORMAL_PICKUP_MAIN_FACTOR_ISLAND if is_island_mode else NORMAL_PICKUP_MAIN_FACTOR

    logger.info(f"({scenario_id}) Usando parámetros {'de modo isla' if is_island_mode else 'normales'}: "
                f"W_MT={w_mt}, W_TIME={w_time}, TARGET_TMT={target_tmt}, MIN_ALLOWED_INDIVIDUAL_MT={min_allowed_individual_mt}")

    for iteration in range(MAX_ITERATIONS):
        total_main_time = 0.0
        tmt = 0.0
        miscoordination_penalty = 0.0
        current_pair_results = []
        max_neg_mt = 0
        error_pairs_count = 0

        for pair in pairs_info:
            main_relay = pair["main_relay"]
            backup_relay = pair["backup_relay"]
            I_shc_main = pair["I_shc_main"]
            I_shc_backup = pair["I_shc_backup"]
            if main_relay not in relay_settings or backup_relay not in relay_settings:
                continue

            tds_main = relay_settings[main_relay]["TDS"]
            pickup_main = relay_settings[main_relay]["pickup"]
            tds_backup = relay_settings[backup_relay]["TDS"]
            pickup_backup = relay_settings[backup_relay]["pickup"]

            pickup_main_bounded = max(MIN_PICKUP, pickup_main)
            pickup_backup_bounded = max(MIN_PICKUP, pickup_backup)

            main_time = calculate_operation_time(I_shc_main, pickup_main_bounded, tds_main)
            backup_time = calculate_operation_time(I_shc_backup, pickup_backup_bounded, tds_backup)

            delta_t: Optional[float] = None
            mt: Optional[float] = None

            if main_time is None or backup_time is None:
                delta_t = None
                mt = None
                error_pairs_count += 1
            elif main_time >= MAX_TIME or backup_time >= MAX_TIME:
                delta_t = backup_time - main_time if main_time < MAX_TIME and backup_time < MAX_TIME else MAX_TIME
                mt = MAX_TIME * 2
            else:
                delta_t = backup_time - main_time
                mt = delta_t - CTI

            current_pair_results.append({
                "main_time": main_time,
                "backup_time": backup_time,
                "delta_t": delta_t,
                "mt": mt,
                "main_relay": main_relay,
                "backup_relay": backup_relay,
                "I_shc_main": I_shc_main,
                "I_shc_backup": I_shc_backup
            })

            if main_time is not None and main_time < MAX_TIME:
                total_main_time += main_time

            if mt is not None and mt < 0 and mt < MAX_TIME:
                tmt += mt
                miscoordination_penalty += mt**2
                max_neg_mt = min(max_neg_mt, mt)

        of = w_mt * miscoordination_penalty + w_time * total_main_time

        if (iteration + 1) % 25 == 0 or iteration == 0 or tmt > target_tmt*1.1:
            logger.info(f"({scenario_id}) Iter {iteration+1}/{MAX_ITERATIONS}: OF={of:.4f}, TMT={tmt:.4f}, MaxNegMT={max_neg_mt:.4f}, Errors={error_pairs_count}")

        if tmt >= target_tmt and max_neg_mt >= min_allowed_individual_mt:
            logger.info(f"({scenario_id}) Convergencia alcanzada (TMT={tmt:.4f} >= {target_tmt}, MaxNegMT={max_neg_mt:.4f} >= {min_allowed_individual_mt}).")
            break

        adjustments_made = False
        next_relay_settings = copy.deepcopy(relay_settings)

        for pair_res in current_pair_results:
            if pair_res["mt"] is None or pair_res["mt"] >= MAX_TIME:
                continue
            if pair_res["mt"] < 0:
                main_relay = pair_res["main_relay"]
                backup_relay = pair_res["backup_relay"]
                I_shc_main = pair_res["I_shc_main"]
                I_shc_backup = pair_res["I_shc_backup"]
                mt_val = pair_res["mt"]

                tds_main_curr = next_relay_settings[main_relay]["TDS"]
                tds_backup_curr = next_relay_settings[backup_relay]["TDS"]
                pickup_main_curr = next_relay_settings[main_relay]["pickup"]
                pickup_backup_curr = next_relay_settings[backup_relay]["pickup"]

                new_tds_backup = tds_backup_curr
                new_pickup_backup = pickup_backup_curr
                new_tds_main = tds_main_curr
                new_pickup_main = pickup_main_curr

                if mt_val < AGGRESSIVE_MT_THRESHOLD:
                    new_tds_backup = min(MAX_TDS, tds_backup_curr * aggressive_tds_backup_factor)
                    if abs(new_tds_backup - MAX_TDS) < 1e-6:
                        max_allowed_pickup_backup = I_shc_backup * MAX_PICKUP_FACTOR
                        new_pickup_backup = min(max_allowed_pickup_backup, max(MIN_PICKUP, pickup_backup_curr * aggressive_pickup_backup_factor))
                    new_tds_main = max(MIN_TDS, tds_main_curr * aggressive_tds_main_factor)
                else:
                    new_tds_backup = min(MAX_TDS, max(MIN_TDS, tds_backup_curr + normal_tds_backup_add))
                    new_tds_main = min(MAX_TDS, max(MIN_TDS, tds_main_curr - normal_tds_main_sub))

                next_relay_settings[backup_relay]["TDS"] = min(MAX_TDS, max(MIN_TDS, new_tds_backup))
                next_relay_settings[main_relay]["TDS"] = min(MAX_TDS, max(MIN_TDS, new_tds_main))
                max_pickup_b = I_shc_backup * MAX_PICKUP_FACTOR
                max_pickup_m = I_shc_main * MAX_PICKUP_FACTOR
                next_relay_settings[backup_relay]["pickup"] = min(max_pickup_b, max(MIN_PICKUP, new_pickup_backup))
                next_relay_settings[main_relay]["pickup"] = min(max_pickup_m, max(MIN_PICKUP, new_pickup_main))

                adjustments_made = True

        if adjustments_made:
            relay_settings = next_relay_settings
        elif iteration > 15:
            logger.info(f"({scenario_id}) No se realizaron ajustes en iteración {iteration + 1}. Deteniendo.")
            break

        if abs(tmt - last_tmt) < CONVERGENCE_THRESHOLD_TMT:
            no_improvement_streak += 1
        else:
            no_improvement_streak = 0
        last_tmt = tmt

        if no_improvement_streak >= 25:
            logger.warning(f"({scenario_id}) TMT no mejora significativamente ({no_improvement_streak} iteraciones). Deteniendo.")
            break
    else:
        logger.warning(f"({scenario_id}) La optimización no convergió después de {MAX_ITERATIONS} iteraciones.")
        logger.warning(f"({scenario_id}) Estado final: TMT={tmt:.4f}, MaxNegMT={max_neg_mt:.4f}, Errors={error_pairs_count}")

    formatted_settings = {}
    relays_max_ishc = {}
    for relay in relays_in_scenario:
        max_ishc = 0
        for p in pairs_info:
            if p['main_relay'] == relay:
                max_ishc = max(max_ishc, p.get('I_shc_main', 0))
            if p['backup_relay'] == relay:
                max_ishc = max(max_ishc, p.get('I_shc_backup', 0))
        relays_max_ishc[relay] = max_ishc if max_ishc > 0 else (default_pickup / MIN_PICKUP)

    for relay, settings in relay_settings.items():
        final_pickup = settings['pickup']
        max_ishc_relay = relays_max_ishc.get(relay, default_pickup / MIN_PICKUP)
        final_pickup_bounded = min(max_ishc_relay * MAX_PICKUP_FACTOR, max(MIN_PICKUP, final_pickup))
        final_tds_bounded = min(MAX_TDS, max(MIN_TDS, settings['TDS']))
        formatted_settings[relay] = {
            "TDS": float(f"{final_tds_bounded:.5f}"),
            "pickup": float(f"{final_pickup_bounded:.5f}")
        }
    logger.info(f"--- Optimización finalizada para {scenario_id} ---")
    return formatted_settings

if __name__ == "__main__":
    logger.info("--- Iniciando Script de Optimización de Ajustes de Relés (v8 - Adjusted Island Mode Optimization) ---")
    if not os.path.exists(RELAY_PAIRS_PATH):
        logger.error(f"¡Error Crítico! No se encontró el archivo de entrada: {RELAY_PAIRS_PATH}")
        raise SystemExit("Archivo de pares de relés no encontrado.")

    relay_pairs_data = load_json_file(RELAY_PAIRS_PATH)
    if relay_pairs_data is None:
        raise SystemExit("Error crítico al cargar archivo de pares.")

    scenario_data_map = group_data_by_scenario(relay_pairs_data)
    if not scenario_data_map:
        raise SystemExit("Error crítico al procesar datos de entrada. No se encontraron escenarios o pares válidos.")

    all_optimized_settings = {}
    successful_scenarios = 0
    failed_scenarios = 0
    for scenario_id, scenario_data in scenario_data_map.items():
        if not scenario_data.get('relays') or not scenario_data.get('pairs_info'):
            logger.warning(f"Omitiendo escenario '{scenario_id}': No contiene relés o pares de información válidos después del procesamiento inicial.")
            failed_scenarios += 1
            continue
        try:
            optimized_settings_for_scenario = run_scenario_optimization(
                scenario_id,
                scenario_data['pairs_info'],
                scenario_data['initial_settings'],
                scenario_data['relays'],
                scenario_data['is_island_mode']
            )
            if optimized_settings_for_scenario:
                all_optimized_settings[scenario_id] = optimized_settings_for_scenario
                successful_scenarios += 1
            else:
                logger.warning(f"La optimización no produjo resultados para el escenario: {scenario_id}")
                failed_scenarios += 1
        except Exception as e:
            logger.error(f"Error inesperado durante optimización del escenario {scenario_id}: {e}", exc_info=True)
            failed_scenarios += 1

    output_list = []
    if all_optimized_settings:
        logger.info(f"Optimización completada. Escenarios exitosos: {successful_scenarios}, Escenarios fallidos/omitidos: {failed_scenarios}.")
        logger.info("Formateando resultados optimizados en la estructura de lista deseada...")
        for scenario_id, optimized_settings in all_optimized_settings.items():
            current_timestamp = datetime.now(timezone.utc).isoformat(timespec='microseconds').replace('+00:00', 'Z')
            list_entry = {
                "scenario_id": scenario_id,
                "timestamp": current_timestamp,
                "relay_values": optimized_settings
            }
            output_list.append(list_entry)
        logger.info(f"Formato de lista creado con {len(output_list)} escenarios optimizados.")

        try:
            output_dir = os.path.dirname(OPTIMIZED_SETTINGS_OUTPUT_PATH)
            if output_dir:
                os.makedirs(output_dir, exist_ok=True)
            with open(OPTIMIZED_SETTINGS_OUTPUT_PATH, 'w') as file:
                json.dump(output_list, file, indent=2, allow_nan=False)
            logger.info(f"Archivo con ajustes optimizados (v8, adjusted island mode optimization) guardado en: {OPTIMIZED_SETTINGS_OUTPUT_PATH}")
        except Exception as e:
            logger.error(f"Error al guardar el archivo de salida {OPTIMIZED_SETTINGS_OUTPUT_PATH}: {e}")
    else:
        logger.error(f"La optimización falló o no produjo resultados para ningún escenario ({successful_scenarios} exitosos, {failed_scenarios} fallidos/omitidos). No se guardó ningún archivo.")

    logger.info("--- Script de Optimización Finalizado ---")

2025-04-20 13:43:53,558 - INFO - --- Iniciando Script de Optimización de Ajustes de Relés (v8 - Adjusted Island Mode Optimization) ---
2025-04-20 13:43:53,609 - INFO - Archivo cargado: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs.json
2025-04-20 13:43:53,628 - INFO - Datos agrupados por escenario. Pares procesados: 6720, Pares omitidos: 80
2025-04-20 13:43:53,629 - INFO - --- Iniciando optimización para scenario_1 ---
2025-04-20 13:43:53,629 - INFO - (scenario_1) Usando parámetros normales: W_MT=15.0, W_TIME=0.1, TARGET_TMT=-0.005, MIN_ALLOWED_INDIVIDUAL_MT=-0.009
2025-04-20 13:43:53,634 - INFO - (scenario_1) Iter 1/250: OF=136.4166, TMT=-23.8825, MaxNegMT=-0.6159, Errors=0
2025-04-20 13:43:53,657 - INFO - (scenario_1) Iter 25/250: OF=25.0857, TMT=-6.2208, MaxNegMT=-0.4648, Errors=0
2025-04-20 13:43:53,671 - INFO - (scenario_1) Iter 50/250: OF=13.6421, TMT=-3.0775, MaxNegMT=-0.2350, Errors=0
2025-04-20 13:43:53,693 - INFO - (sce

In [2]:
import json
import numpy as np
import logging
import os
import math
import copy
import random
import string
from datetime import datetime, timezone
from typing import Dict, List, Any, Optional, Tuple, Set

# --- Configuration ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# --- File Paths ---
# PLEASE UPDATE THESE PATHS TO YOUR ACTUAL FILE LOCATIONS
RELAY_PAIRS_PATH = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs.json"
OPTIMIZED_SETTINGS_OUTPUT_PATH = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/raw/optimized_relay_values_v10.json" # Changed version in output name

# --- Constants ---
K = 0.14
N = 0.02
CTI = 0.2
# Optimization Bounds and Parameters
MIN_TDS = 0.05
MAX_TDS = 1.0 # Corrected: Should be float
MIN_PICKUP = 0.05
MAX_PICKUP_FACTOR = 0.7
MAX_TIME = 20.0
MAX_ITERATIONS = 250
# Convergence Targets (Normal Mode)
TARGET_TMT = -0.005
MIN_ALLOWED_INDIVIDUAL_MT = -0.009
# Convergence Targets (Island Mode)
TARGET_TMT_ISLAND = -0.05  # More relaxed for island mode
MIN_ALLOWED_INDIVIDUAL_MT_ISLAND = -0.02  # More relaxed for island mode
CONVERGENCE_THRESHOLD_TMT = 0.005

# --- NEW: Objective Function Weights (Based on Interpreted Formula) ---
# OF = alpha1 * sum(t_main^2) + alpha2 * sum(max(0, CTI - delta_t)^2)
ALPHA_1_NORMAL = 0.1  # Weight for sum of squared main times (Normal)
ALPHA_2_NORMAL = 15.0 # Weight for sum of squared coordination violations (Normal)
ALPHA_1_ISLAND = 0.5  # Weight for sum of squared main times (Island) - Prioritize speed
ALPHA_2_ISLAND = 5.0  # Weight for sum of squared coordination violations (Island) - Less strict on coordination

# Adjustment Step Sizes & Factors (Normal Mode) - KEEPING THESE
AGGRESSIVE_MT_THRESHOLD = -CTI * 0.75
AGGRESSIVE_TDS_BACKUP_FACTOR = 1.15
AGGRESSIVE_TDS_MAIN_FACTOR = 0.90
AGGRESSIVE_PICKUP_BACKUP_FACTOR = 1.05 # Pickup adjustments were missing, added based on context
NORMAL_TDS_BACKUP_ADD = 0.02
NORMAL_TDS_MAIN_SUB = 0.01
NORMAL_PICKUP_BACKUP_FACTOR = 1.01 # Pickup adjustments were missing, added based on context
NORMAL_PICKUP_MAIN_FACTOR = 0.99   # Pickup adjustments were missing, added based on context

# Adjustment Step Sizes & Factors (Island Mode - More Conservative) - KEEPING THESE
AGGRESSIVE_TDS_BACKUP_FACTOR_ISLAND = 1.05
AGGRESSIVE_TDS_MAIN_FACTOR_ISLAND = 0.95
AGGRESSIVE_PICKUP_BACKUP_FACTOR_ISLAND = 1.02 # Pickup adjustments were missing, added based on context
NORMAL_TDS_BACKUP_ADD_ISLAND = 0.01
NORMAL_TDS_MAIN_SUB_ISLAND = 0.005
NORMAL_PICKUP_BACKUP_FACTOR_ISLAND = 1.005 # Pickup adjustments were missing, added based on context
NORMAL_PICKUP_MAIN_FACTOR_ISLAND = 0.995  # Pickup adjustments were missing, added based on context

# Island Mode Detection
ISLAND_MODE_TMT_THRESHOLD = -10.0  # Threshold to identify island mode scenarios
EXCLUDED_RELAYS = {f"R{i}" for i in [2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
                                     37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52]}
# --- Helper Functions ---
def load_json_file(file_path: str) -> Optional[Any]:
    try:
        with open(file_path, 'r') as file: data = json.load(file)
        logger.info(f"Archivo cargado: {file_path}"); return data
    except FileNotFoundError: logger.error(f"No encontrado: {file_path}"); return None
    except json.JSONDecodeError as e: logger.error(f"JSON inválido: {file_path}: {e}"); return None
    except Exception as e: logger.error(f"Error carga {file_path}: {e}"); return None

def calculate_operation_time(I_shc: float, I_pi: float, TDS: float) -> Optional[float]:
    # Ensure inputs are valid numbers before calculations
    if not all(isinstance(x, (int, float)) for x in [I_shc, I_pi, TDS]):
        # logger.warning(f"Invalid input type in calc_op_time: I_shc={I_shc}, I_pi={I_pi}, TDS={TDS}")
        return None
    if I_pi <= 0 or I_shc <= 0 or TDS < MIN_TDS or TDS > MAX_TDS:
        # logger.debug(f"Input out of bounds: I_pi={I_pi}, I_shc={I_shc}, TDS={TDS}")
        return None # Return None for invalid bounds, let caller decide penalty

    # Check if pickup is too high relative to fault current
    # Using a small epsilon to avoid floating point issues near the boundary
    if I_pi > I_shc * MAX_PICKUP_FACTOR + 1e-9 :
        # logger.debug(f"Pickup too high: I_pi={I_pi} > I_shc*factor={I_shc * MAX_PICKUP_FACTOR}. Returning MAX_TIME")
        return MAX_TIME

    M = I_shc / I_pi
    # If M <= 1, the relay theoretically never picks up.
    if M <= 1.0:
        # logger.debug(f"M <= 1 (M={M:.4f}). Returning MAX_TIME")
        return MAX_TIME

    try:
        # Standard inverse time curve formula
        denominator = M**N - 1
        # Avoid division by zero or very small numbers if M is extremely close to 1
        if abs(denominator) < 1e-9:
            # logger.debug(f"Denominator too small (M={M:.4f}). Returning MAX_TIME")
            return MAX_TIME # Treat as non-operating scenario

        time = TDS * (K / denominator)

        # Check for non-finite results or zero/negative time (shouldn't happen with M>1)
        if not np.isfinite(time) or time <= 0:
            # logger.warning(f"Non-finite or non-positive time calculated (time={time}). Returning None")
            return None # Indicate calculation failure

        # Return calculated time, capped by MAX_TIME
        return min(time, MAX_TIME)

    except (OverflowError, ValueError) as e:
        # logger.error(f"Numeric error in calc_op_time (M={M:.4f}, TDS={TDS}): {e}")
        return None # Indicate calculation failure
    except Exception as e:
        # Catch any other unexpected exceptions
        logger.error(f"Unexpected exception in calc_op_time (I_shc={I_shc}, I_pi={I_pi}, TDS={TDS}): {e}")
        return None

def group_data_by_scenario(relay_pairs_data: List[Dict]) -> Dict[str, Dict[str, Any]]:
    scenario_map: Dict[str, Dict[str, Any]] = {}
    processed_pairs_count = 0
    skipped_pairs_count = 0
    island_scenarios = set()

    # First pass: Identify island mode scenarios based on TMT from input file
    # Note: This TMT is from the *input* data, just used for mode detection strategy
    for pair_entry in relay_pairs_data:
        scenario_id = pair_entry.get("scenario_id")
        tmt_input = pair_entry.get("TMT") # Assuming TMT might be in the input pair data
        if scenario_id and isinstance(tmt_input, (int, float)) and tmt_input < ISLAND_MODE_TMT_THRESHOLD:
            if scenario_id not in island_scenarios:
                 logger.info(f"Scenario {scenario_id} initially flagged as potential island mode (Input TMT={tmt_input:.4f} < {ISLAND_MODE_TMT_THRESHOLD})")
                 island_scenarios.add(scenario_id)

    # Second pass: Process pairs
    for i, pair_entry in enumerate(relay_pairs_data):
        scenario_id = pair_entry.get("scenario_id")
        main_relay_info = pair_entry.get('main_relay')
        backup_relay_info = pair_entry.get('backup_relay')

        if not scenario_id or not isinstance(main_relay_info, dict) or not isinstance(backup_relay_info, dict):
            # logger.warning(f"Skipping entry {i}: Invalid format or missing data (scenario_id, main_relay, backup_relay)")
            skipped_pairs_count += 1
            continue

        is_island = scenario_id in island_scenarios # Check if flagged

        if scenario_id not in scenario_map:
            scenario_map[scenario_id] = {"pairs_info": [], "initial_settings": {}, "relays": set(), "is_island_mode": is_island}
        elif is_island: # Ensure flag is set if any pair triggered it
             scenario_map[scenario_id]["is_island_mode"] = True


        main_relay = main_relay_info.get('relay')
        backup_relay = backup_relay_info.get('relay')
        I_shc_main = main_relay_info.get('Ishc')
        I_shc_backup = backup_relay_info.get('Ishc')

        # Skip pairs involving EXCLUDED_RELAYS *only if* in island mode
        if scenario_map[scenario_id]["is_island_mode"] and (main_relay in EXCLUDED_RELAYS or backup_relay in EXCLUDED_RELAYS):
            # logger.debug(f"Skipping pair {main_relay}/{backup_relay} in scenario {scenario_id} (Island mode, excluded relay)")
            skipped_pairs_count += 1
            continue

        # Validate essential data for the pair
        if not (main_relay and backup_relay and
                isinstance(I_shc_main, (int, float)) and I_shc_main > 0 and
                isinstance(I_shc_backup, (int, float)) and I_shc_backup > 0):
            # logger.warning(f"Skipping pair {main_relay}/{backup_relay} in scenario {scenario_id}: Missing or invalid relay names or Ishc values.")
            skipped_pairs_count += 1
            continue

        processed_pairs_count += 1
        scenario_map[scenario_id]['relays'].add(main_relay)
        scenario_map[scenario_id]['relays'].add(backup_relay)

        # Store initial settings from the first time a relay appears in the scenario
        initial_settings_scenario = scenario_map[scenario_id]['initial_settings']
        for r_name, r_info in [(main_relay, main_relay_info), (backup_relay, backup_relay_info)]:
            if r_name not in initial_settings_scenario:
                tds = r_info.get('TDS')
                pickup = r_info.get('pick_up') # Name used in input file
                if isinstance(tds, (int, float)) and isinstance(pickup, (int, float)):
                    initial_settings_scenario[r_name] = {
                        'TDS_initial': float(tds),
                        'pickup_initial': float(pickup)
                    }
                # else:
                    # logger.warning(f"Initial settings (TDS/pickup) not found or invalid for relay '{r_name}' in scenario {scenario_id}. Will use defaults during optimization init.")


        scenario_map[scenario_id]['pairs_info'].append({
            "main_relay": main_relay,
            "backup_relay": backup_relay,
            "I_shc_main": float(I_shc_main),
            "I_shc_backup": float(I_shc_backup)
            # "pair_index": i # Optional: Keep track of original index if needed
        })

    logger.info(f"Data grouped by scenario. Scenarios found: {len(scenario_map)}. Pairs processed: {processed_pairs_count}, Pairs skipped: {skipped_pairs_count}")
    # Log island mode detection summary
    final_island_count = sum(1 for sc_data in scenario_map.values() if sc_data['is_island_mode'])
    logger.info(f"Detected {final_island_count} island mode scenarios out of {len(scenario_map)} total.")

    return scenario_map


def run_scenario_optimization(
    scenario_id: str,
    pairs_info: List[Dict],
    initial_settings: Dict[str, Dict[str, float]],
    relays_in_scenario: Set[str],
    is_island_mode: bool
) -> Dict[str, Dict[str, float]]:
    logger.info(f"--- Starting optimization for {scenario_id} {'(ISLAND MODE)' if is_island_mode else '(NORMAL MODE)'} ---")
    if not pairs_info:
        logger.warning(f"({scenario_id}) No valid pairs to optimize. Returning empty settings.")
        return {}

    # --- Initialization ---
    relay_settings = {}
    default_pickup = MIN_PICKUP * 1.5 # A reasonable starting point if initial pickup is missing
    default_tds = MIN_TDS # Start TDS at minimum

    for relay in relays_in_scenario:
        if relay in initial_settings and 'pickup_initial' in initial_settings[relay]:
             # Start with minimum TDS and the initial pickup (or MIN_PICKUP if initial is too low)
             initial_pickup = initial_settings[relay]['pickup_initial']
             relay_settings[relay] = {
                 "TDS": default_tds,
                 "pickup": max(MIN_PICKUP, initial_pickup)
             }
        else:
            # logger.warning(f"({scenario_id}) Initial settings (pickup) not found for relay '{relay}'. Using default pickup: {default_pickup:.4f}")
            relay_settings[relay] = {
                "TDS": default_tds,
                "pickup": default_pickup
             }

    last_tmt = float('inf') # Track TMT from previous iteration
    no_improvement_streak = 0 # Count iterations without significant TMT improvement

    # --- Select Parameters Based on Mode ---
    # Objective Function Weights
    alpha_1 = ALPHA_1_ISLAND if is_island_mode else ALPHA_1_NORMAL
    alpha_2 = ALPHA_2_ISLAND if is_island_mode else ALPHA_2_NORMAL
    # Convergence Targets
    target_tmt = TARGET_TMT_ISLAND if is_island_mode else TARGET_TMT
    min_allowed_individual_mt = MIN_ALLOWED_INDIVIDUAL_MT_ISLAND if is_island_mode else MIN_ALLOWED_INDIVIDUAL_MT
    # Adjustment Factors
    aggressive_tds_backup_factor = AGGRESSIVE_TDS_BACKUP_FACTOR_ISLAND if is_island_mode else AGGRESSIVE_TDS_BACKUP_FACTOR
    aggressive_tds_main_factor = AGGRESSIVE_TDS_MAIN_FACTOR_ISLAND if is_island_mode else AGGRESSIVE_TDS_MAIN_FACTOR
    aggressive_pickup_backup_factor = AGGRESSIVE_PICKUP_BACKUP_FACTOR_ISLAND if is_island_mode else AGGRESSIVE_PICKUP_BACKUP_FACTOR
    normal_tds_backup_add = NORMAL_TDS_BACKUP_ADD_ISLAND if is_island_mode else NORMAL_TDS_BACKUP_ADD
    normal_tds_main_sub = NORMAL_TDS_MAIN_SUB_ISLAND if is_island_mode else NORMAL_TDS_MAIN_SUB
    normal_pickup_backup_factor = NORMAL_PICKUP_BACKUP_FACTOR_ISLAND if is_island_mode else NORMAL_PICKUP_BACKUP_FACTOR
    normal_pickup_main_factor = NORMAL_PICKUP_MAIN_FACTOR_ISLAND if is_island_mode else NORMAL_PICKUP_MAIN_FACTOR

    logger.info(f"({scenario_id}) Using Objective Function: alpha1*sum(t_main^2) + alpha2*sum(max(0, CTI-delta_t)^2)")
    logger.info(f"({scenario_id}) Weights: alpha1={alpha_1}, alpha2={alpha_2}")
    logger.info(f"({scenario_id}) Convergence Targets: TMT >= {target_tmt}, Min MT >= {min_allowed_individual_mt}")
    logger.info(f"({scenario_id}) Adjustment Params: Aggressive MT Threshold={AGGRESSIVE_MT_THRESHOLD:.4f}")


    # --- Optimization Loop ---
    for iteration in range(MAX_ITERATIONS):
        # --- Calculate Performance Metrics for Current Settings ---
        sum_sq_main_time = 0.0
        sum_sq_coord_penalty = 0.0
        tmt = 0.0  # Total Margin Time (Sum of negative MTs) - Still needed for convergence/adjustments
        max_neg_mt = 0.0 # Most negative MT observed - Still needed for convergence
        current_pair_results = [] # Store results for adjustments
        error_pairs_count = 0     # Count pairs with calculation errors
        large_penalty_value_of = (MAX_TIME * 10)**2 # Large penalty added to OF for errors/max_time

        for pair_index, pair in enumerate(pairs_info):
            main_relay = pair["main_relay"]
            backup_relay = pair["backup_relay"]
            I_shc_main = pair["I_shc_main"]
            I_shc_backup = pair["I_shc_backup"]

            # Should not happen if grouping is correct, but check anyway
            if main_relay not in relay_settings or backup_relay not in relay_settings:
                 logger.error(f"({scenario_id}) Iter {iteration+1}: Relay settings missing for pair {main_relay}/{backup_relay}. Skipping pair.")
                 error_pairs_count += 1
                 # Add penalty? Need times to calculate penalty based on delta_t later
                 # Store None to indicate error for this pair
                 current_pair_results.append({
                     "main_time": None, "backup_time": None, "delta_t": None, "mt": None,
                     "main_relay": main_relay, "backup_relay": backup_relay,
                     "I_shc_main": I_shc_main, "I_shc_backup": I_shc_backup
                 })
                 continue

            # Get current settings
            tds_main = relay_settings[main_relay]["TDS"]
            pickup_main = relay_settings[main_relay]["pickup"]
            tds_backup = relay_settings[backup_relay]["TDS"]
            pickup_backup = relay_settings[backup_relay]["pickup"]

            # Pickup should already be bounded, but ensure > MIN_PICKUP for calculation
            pickup_main_calc = max(MIN_PICKUP, pickup_main)
            pickup_backup_calc = max(MIN_PICKUP, pickup_backup)

            # Calculate operation times
            main_time = calculate_operation_time(I_shc_main, pickup_main_calc, tds_main)
            backup_time = calculate_operation_time(I_shc_backup, pickup_backup_calc, tds_backup)

            # Calculate delta_t and mt
            delta_t: Optional[float] = None
            mt: Optional[float] = None

            if main_time is None or backup_time is None:
                # Error in time calculation for main or backup
                # logger.debug(f"({scenario_id}) Iter {iteration+1}, Pair {pair_index}: Calc error. main_time={main_time}, backup_time={backup_time}")
                error_pairs_count += 1
                delta_t = None
                mt = None
                # Penalize objective function terms heavily
                sum_sq_main_time += large_penalty_value_of
                sum_sq_coord_penalty += large_penalty_value_of
            elif main_time >= MAX_TIME and backup_time >= MAX_TIME:
                 # Both relays hit max time - consider this non-coordinated or very slow
                 # logger.debug(f"({scenario_id}) Iter {iteration+1}, Pair {pair_index}: Both MAX_TIME.")
                 delta_t = 0.0 # Arbitrarily set to 0, as difference is undefined/irrelevant
                 mt = 0.0 - CTI # Treat as coordination failure for adjustment? Or just penalize OF? Let's penalize OF.
                 sum_sq_main_time += MAX_TIME**2
                 sum_sq_coord_penalty += (CTI**2) # Penalize as if delta_t was 0
            else:
                 # At least one relay operates below MAX_TIME
                 delta_t = backup_time - main_time
                 mt = delta_t - CTI
                 # logger.debug(f"({scenario_id}) Iter {iteration+1}, Pair {pair_index}: main_t={main_time:.4f}, backup_t={backup_time:.4f}, delta_t={delta_t:.4f}, mt={mt:.4f}")

                 # Accumulate for Objective Function
                 if main_time < MAX_TIME:
                     sum_sq_main_time += main_time**2
                 else: # main_time is MAX_TIME (backup_time must be < MAX_TIME here)
                     sum_sq_main_time += MAX_TIME**2

                 coordination_violation = max(0.0, CTI - delta_t)
                 sum_sq_coord_penalty += coordination_violation**2

                 # Accumulate TMT (sum of negative MTs) and track most negative MT
                 if mt < 0:
                     tmt += mt
                     max_neg_mt = min(max_neg_mt, mt)

            # Store results for adjustments phase
            current_pair_results.append({
                "main_time": main_time,
                "backup_time": backup_time,
                "delta_t": delta_t,
                "mt": mt,
                "main_relay": main_relay,
                "backup_relay": backup_relay,
                "I_shc_main": I_shc_main,
                "I_shc_backup": I_shc_backup
            })

        # Calculate the overall objective function for this iteration
        of = alpha_1 * sum_sq_main_time + alpha_2 * sum_sq_coord_penalty

        # --- Logging ---
        if (iteration + 1) % 25 == 0 or iteration == 0:
            logger.info(f"({scenario_id}) Iter {iteration+1}/{MAX_ITERATIONS}: OF={of:.4f}, TMT={tmt:.4f}, MaxNegMT={max_neg_mt:.4f}, Errors={error_pairs_count}")

        # --- Convergence Check ---
        # Check if TMT is non-negative enough AND the single worst miscoordination is acceptable
        if tmt >= target_tmt and max_neg_mt >= min_allowed_individual_mt:
            logger.info(f"({scenario_id}) Convergence target met at iteration {iteration+1} (TMT={tmt:.4f} >= {target_tmt}, MaxNegMT={max_neg_mt:.4f} >= {min_allowed_individual_mt}).")
            break

        # --- Adjust Relay Settings ---
        adjustments_made = False
        next_relay_settings = copy.deepcopy(relay_settings) # Work on a copy

        # Iterate through pairs *that had a coordination issue*
        for pair_res in current_pair_results:
            # Skip if error or already coordinated (or MAX_TIME special cases handled by OF penalty)
            if pair_res["mt"] is None or pair_res["mt"] >= 0:
                continue

            # We have a miscoordination: mt < 0
            main_relay = pair_res["main_relay"]
            backup_relay = pair_res["backup_relay"]
            I_shc_main = pair_res["I_shc_main"] # Needed for pickup bounding
            I_shc_backup = pair_res["I_shc_backup"] # Needed for pickup bounding
            mt_val = pair_res["mt"]

            # Get current settings from the 'next_relay_settings' we are modifying
            tds_main_curr = next_relay_settings[main_relay]["TDS"]
            tds_backup_curr = next_relay_settings[backup_relay]["TDS"]
            pickup_main_curr = next_relay_settings[main_relay]["pickup"]
            pickup_backup_curr = next_relay_settings[backup_relay]["pickup"]

            # Initialize new values to current ones
            new_tds_backup = tds_backup_curr
            new_pickup_backup = pickup_backup_curr
            new_tds_main = tds_main_curr
            new_pickup_main = pickup_main_curr

            # Apply adjustment logic based on severity of miscoordination (mt_val)
            if mt_val < AGGRESSIVE_MT_THRESHOLD:
                # Aggressive adjustment: Increase backup TDS/Pickup significantly, decrease main TDS
                new_tds_backup = tds_backup_curr * aggressive_tds_backup_factor
                # Only increase backup pickup aggressively if TDS is already maxed out
                if abs(new_tds_backup - MAX_TDS) < 1e-6 or new_tds_backup > MAX_TDS:
                    new_pickup_backup = pickup_backup_curr * aggressive_pickup_backup_factor
                new_tds_main = tds_main_curr * aggressive_tds_main_factor
                # Aggressively reducing main pickup is generally not done, focus on TDS
            else:
                # Normal adjustment: Small increase for backup TDS/Pickup, small decrease for main TDS/Pickup
                new_tds_backup = tds_backup_curr + normal_tds_backup_add
                new_pickup_backup = pickup_backup_curr * normal_pickup_backup_factor # Use factor for pickup
                new_tds_main = tds_main_curr - normal_tds_main_sub
                new_pickup_main = pickup_main_curr * normal_pickup_main_factor # Use factor for pickup

            # --- Apply Bounds to Adjusted Settings ---
            # TDS Bounds
            final_tds_backup = min(MAX_TDS, max(MIN_TDS, new_tds_backup))
            final_tds_main = min(MAX_TDS, max(MIN_TDS, new_tds_main))

            # Pickup Bounds (Min: MIN_PICKUP, Max: I_shc * MAX_PICKUP_FACTOR)
            max_allowed_pickup_backup = I_shc_backup * MAX_PICKUP_FACTOR
            final_pickup_backup = min(max_allowed_pickup_backup, max(MIN_PICKUP, new_pickup_backup))

            max_allowed_pickup_main = I_shc_main * MAX_PICKUP_FACTOR
            final_pickup_main = min(max_allowed_pickup_main, max(MIN_PICKUP, new_pickup_main))

            # Update the settings for the next iteration *if they changed*
            if abs(final_tds_backup - tds_backup_curr) > 1e-6 or \
               abs(final_pickup_backup - pickup_backup_curr) > 1e-6 or \
               abs(final_tds_main - tds_main_curr) > 1e-6 or \
               abs(final_pickup_main - pickup_main_curr) > 1e-6:
                next_relay_settings[backup_relay]["TDS"] = final_tds_backup
                next_relay_settings[backup_relay]["pickup"] = final_pickup_backup
                next_relay_settings[main_relay]["TDS"] = final_tds_main
                next_relay_settings[main_relay]["pickup"] = final_pickup_main
                adjustments_made = True
                # logger.debug(f"({scenario_id}) Iter {iteration+1}: Adjusted pair {main_relay}(M)/{backup_relay}(B) for MT={mt_val:.4f}")
                # logger.debug(f"  M: TDS {tds_main_curr:.4f}->{final_tds_main:.4f}, Pkup {pickup_main_curr:.4f}->{final_pickup_main:.4f}")
                # logger.debug(f"  B: TDS {tds_backup_curr:.4f}->{final_tds_backup:.4f}, Pkup {pickup_backup_curr:.4f}->{final_pickup_backup:.4f}")


        # --- Update Settings or Check for Stalling ---
        if adjustments_made:
            relay_settings = next_relay_settings # Adopt the adjusted settings
        elif iteration > 15: # Start checking for stalling after initial iterations
            logger.info(f"({scenario_id}) No adjustments made in iteration {iteration + 1}. Stopping early.")
            break # No adjustments needed, likely converged or stuck

        # Check if TMT has stalled (not improving significantly)
        if abs(tmt - last_tmt) < CONVERGENCE_THRESHOLD_TMT:
            no_improvement_streak += 1
        else:
            no_improvement_streak = 0 # Reset streak if TMT changed enough

        last_tmt = tmt # Store TMT for next iteration's comparison

        if no_improvement_streak >= 25: # Stop if TMT hasn't improved for 25 iterations
            logger.warning(f"({scenario_id}) TMT has stalled for {no_improvement_streak} iterations (Last TMT={tmt:.4f}). Stopping early.")
            break
    # --- End of Optimization Loop ---

    else:
        # This block executes if the loop finished without a 'break' (i.e., max iterations reached)
        logger.warning(f"({scenario_id}) Optimization did not converge within {MAX_ITERATIONS} iterations.")
        logger.warning(f"({scenario_id}) Final state: OF={of:.4f}, TMT={tmt:.4f}, MaxNegMT={max_neg_mt:.4f}, Errors={error_pairs_count}")

    # --- Format Final Results ---
    formatted_settings = {}
    # Calculate max Ishc seen by each relay across all its pairs in this scenario
    # This is needed for a final bounding check on pickup, although adjustments should handle it.
    relays_max_ishc = {relay: 0.0 for relay in relays_in_scenario}
    for p in pairs_info:
        if p['main_relay'] in relays_max_ishc:
             relays_max_ishc[p['main_relay']] = max(relays_max_ishc[p['main_relay']], p.get('I_shc_main', 0))
        if p['backup_relay'] in relays_max_ishc:
             relays_max_ishc[p['backup_relay']] = max(relays_max_ishc[p['backup_relay']], p.get('I_shc_backup', 0))

    # Apply final bounds and format output
    for relay, settings in relay_settings.items():
        final_tds = settings['TDS']
        final_pickup = settings['pickup']

        # Final safety bound for pickup based on max Ishc
        max_ishc_relay = relays_max_ishc.get(relay, 0)
        if max_ishc_relay <= 0: # Should not happen if input data is good
             # logger.warning(f"({scenario_id}) Max Ishc for relay {relay} is zero or negative. Using default pickup for bounding.")
             # Use a value derived from default pickup if Ishc is missing/zero
             max_ishc_relay = default_pickup / MAX_PICKUP_FACTOR if MAX_PICKUP_FACTOR > 0 else default_pickup

        max_allowed_pickup = max_ishc_relay * MAX_PICKUP_FACTOR
        final_pickup_bounded = min(max_allowed_pickup, max(MIN_PICKUP, final_pickup))

        # Final bound for TDS
        final_tds_bounded = min(MAX_TDS, max(MIN_TDS, final_tds))

        formatted_settings[relay] = {
            # Format to 5 decimal places as float
            "TDS": float(f"{final_tds_bounded:.5f}"),
            "pickup": float(f"{final_pickup_bounded:.5f}")
        }

    logger.info(f"--- Optimization finished for {scenario_id} ---")
    return formatted_settings

# --- Main Execution ---
if __name__ == "__main__":
    logger.info("--- Starting Relay Settings Optimization Script (v9 - New Objective Function) ---")

    # --- Load Input Data ---
    if not os.path.exists(RELAY_PAIRS_PATH):
        logger.error(f"CRITICAL ERROR! Input file not found: {RELAY_PAIRS_PATH}")
        raise SystemExit("Relay pairs input file not found.")

    relay_pairs_data = load_json_file(RELAY_PAIRS_PATH)
    if relay_pairs_data is None:
        raise SystemExit("Critical error loading relay pairs file.")
    if not isinstance(relay_pairs_data, list):
         raise SystemExit("Critical error: Input data is not a JSON list.")

    # --- Group Data and Detect Modes ---
    scenario_data_map = group_data_by_scenario(relay_pairs_data)
    if not scenario_data_map:
        raise SystemExit("Critical error processing input data. No valid scenarios or pairs found.")

    # --- Run Optimization for Each Scenario ---
    all_optimized_settings = {}
    successful_scenarios = 0
    failed_scenarios = 0

    # Sort scenarios for potentially more consistent run order (optional)
    sorted_scenario_ids = sorted(scenario_data_map.keys())

    for scenario_id in sorted_scenario_ids:
        scenario_data = scenario_data_map[scenario_id]
        # Double-check if the scenario has necessary info after grouping
        if not scenario_data.get('relays') or not scenario_data.get('pairs_info'):
            logger.warning(f"Skipping scenario '{scenario_id}': Contains no valid relays or pairs after processing.")
            failed_scenarios += 1
            continue
        try:
            optimized_settings_for_scenario = run_scenario_optimization(
                scenario_id,
                scenario_data['pairs_info'],
                scenario_data['initial_settings'],
                scenario_data['relays'],
                scenario_data['is_island_mode'] # Pass the detected mode
            )
            if optimized_settings_for_scenario:
                all_optimized_settings[scenario_id] = optimized_settings_for_scenario
                successful_scenarios += 1
            else:
                logger.warning(f"Optimization produced no results for scenario: {scenario_id}")
                failed_scenarios += 1
        except Exception as e:
            logger.error(f"Unexpected error during optimization of scenario {scenario_id}: {e}", exc_info=True)
            failed_scenarios += 1

    # --- Save Results ---
    output_list = []
    if all_optimized_settings:
        logger.info(f"Optimization completed. Successful scenarios: {successful_scenarios}, Failed/Skipped scenarios: {failed_scenarios}.")
        logger.info("Formatting optimized results into the desired list structure...")

        for scenario_id, optimized_settings in all_optimized_settings.items():
            # Get current timestamp in ISO format with Z for UTC
            current_timestamp = datetime.now(timezone.utc).isoformat(timespec='microseconds').replace('+00:00', 'Z')
            list_entry = {
                "scenario_id": scenario_id,
                "timestamp": current_timestamp,
                "relay_values": optimized_settings # Already formatted dict
            }
            output_list.append(list_entry)
        logger.info(f"List format created with {len(output_list)} optimized scenarios.")

        try:
            # Ensure output directory exists
            output_dir = os.path.dirname(OPTIMIZED_SETTINGS_OUTPUT_PATH)
            if output_dir: # Check if path includes a directory
                os.makedirs(output_dir, exist_ok=True) # Create directory if it doesn't exist

            # Write the list of dictionaries to the JSON file
            with open(OPTIMIZED_SETTINGS_OUTPUT_PATH, 'w') as file:
                # Use allow_nan=False to ensure strict JSON compatibility
                json.dump(output_list, file, indent=2, allow_nan=False)
            logger.info(f"Optimized settings file (v9, new OF) saved successfully to: {OPTIMIZED_SETTINGS_OUTPUT_PATH}")
        except IOError as e:
            logger.error(f"Error writing output file {OPTIMIZED_SETTINGS_OUTPUT_PATH}: {e}")
        except Exception as e:
             logger.error(f"Unexpected error saving the output file {OPTIMIZED_SETTINGS_OUTPUT_PATH}: {e}")
    else:
        logger.error(f"Optimization failed or produced no results for any scenario ({successful_scenarios} successful, {failed_scenarios} failed/skipped). No output file was saved.")

    logger.info("--- Optimization Script Finished ---")

2025-04-20 20:07:24,296 - INFO - --- Starting Relay Settings Optimization Script (v9 - New Objective Function) ---
2025-04-20 20:07:24,356 - INFO - Archivo cargado: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs.json
2025-04-20 20:07:24,514 - INFO - Data grouped by scenario. Scenarios found: 68. Pairs processed: 6720, Pairs skipped: 80
2025-04-20 20:07:24,515 - INFO - Detected 0 island mode scenarios out of 68 total.
2025-04-20 20:07:24,519 - INFO - --- Starting optimization for scenario_1 (NORMAL MODE) ---
2025-04-20 20:07:24,520 - INFO - (scenario_1) Using Objective Function: alpha1*sum(t_main^2) + alpha2*sum(max(0, CTI-delta_t)^2)
2025-04-20 20:07:24,520 - INFO - (scenario_1) Weights: alpha1=0.1, alpha2=15.0
2025-04-20 20:07:24,521 - INFO - (scenario_1) Convergence Targets: TMT >= -0.005, Min MT >= -0.009
2025-04-20 20:07:24,523 - INFO - (scenario_1) Adjustment Params: Aggressive MT Threshold=-0.1500
2025-04-20 20:07:24,529 - I